In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
val model = Network("model")

// Create nodes in tandem configuration
val delay = Delay(model, "Delay")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)

// Create two closed job classes with equal populations
val jobclass1 = ClosedClass(model, "Class1", 10, delay, 0)  // 10 jobs
val jobclass2 = ClosedClass(model, "Class2", 10, delay, 0)  // 10 jobs

In [ ]:
// Set identical service processes for both classes
// Delay station
delay.setService(jobclass1, Exp.fitMean(1.0))  // mean = 1
delay.setService(jobclass2, Exp.fitMean(1.0))  // mean = 1

// Queue1 - moderate service rate
queue1.setService(jobclass1, Exp.fitMean(1.5))  // mean = 1.5
queue1.setService(jobclass2, Exp.fitMean(1.5))  // mean = 1.5

// Queue2 - slower service rate (bottleneck)
queue2.setService(jobclass1, Exp.fitMean(3.0))  // mean = 3.0
queue2.setService(jobclass2, Exp.fitMean(3.0))  // mean = 3.0

In [ ]:
// Set tandem routing for both classes using serialRouting
val P = model.initRoutingMatrix()
val nodes = arrayOf(delay, queue1, queue2)

// Use serial routing for both classes (cyclic tandem: Delay -> Queue1 -> Queue2 -> Delay -> ...)
val serialRoute1 = Network.serialRouting(nodes)
val serialRoute2 = Network.serialRouting(nodes)

// Set routing for Class1
for (i in 0 until nodes.size) {
    for (j in 0 until nodes.size) {
        P.set(jobclass1, jobclass1, nodes[i], nodes[j], serialRoute1.get(i, j))
    }
}

// Set routing for Class2
for (i in 0 until nodes.size) {
    for (j in 0 until nodes.size) {
        P.set(jobclass2, jobclass2, nodes[i], nodes[j], serialRoute2.get(i, j))
    }
}

model.link(P)

In [ ]:
// Solve with MVA (following the original MATLAB example approach)val solver = MVA(model)val avgTable = solver.getAvgTable()avgTable.print()

This multi-class tandem model demonstrates:

1. **Tandem Configuration**:
   - Jobs flow sequentially: Delay → Queue1 → Queue2 → Delay
   - Both classes follow identical routing patterns
   - No class switching - classes maintain their identity

2. **Resource Competition**:
   - 20 total jobs (10 per class) compete for the same servers
   - Both classes have identical service requirements
   - FCFS scheduling treats classes equally at each queue

3. **Service Rate Progression**:
   - **Delay**: Fast service (rate = 1.0)
   - **Queue1**: Moderate service (rate = 1/1.5 ≈ 0.67)
   - **Queue2**: Slow service (rate = 1/3.0 ≈ 0.33) - **bottleneck**

4. **Performance Characteristics**:
   - Queue2 becomes the bottleneck due to slowest service
   - Both classes experience identical performance (symmetric system)
   - Higher queue lengths expected at Queue2
   - Response times increase through the tandem stages

5. **MVA Solution**:
   - Mean Value Analysis provides exact analytical results
   - Efficient computation for this class of models
   - Demonstrates the impact of bottleneck stations

The model represents manufacturing or service systems where:
- Multiple customer types follow the same processing steps
- Each stage has different processing capabilities
- Bottleneck identification is crucial for capacity planning
- Fair scheduling (FCFS) ensures equal treatment